In [57]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import functional as TF
import random
from tqdm import tqdm # 用於顯示進度條的套件
import matplotlib.pyplot as plt

In [58]:
# prompt: 連接雲端

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# prompt: 解壓縮/content/drive/MyDrive/大專生計畫/data224_224.rar 並存於命名為data_set的資料夾

# 檢查資料夾是否存在，如果存在則刪除並重新建立
if not os.path.exists('/content/drive/MyDrive/大專生計畫/data_set'):
  os.makedirs('/content/drive/MyDrive/大專生計畫/data_set')
  !unrar x /content/drive/MyDrive/大專生計畫/data224_224.rar /content/drive/MyDrive/大專生計畫/data_set


In [60]:
# 設定參數
IMAGE_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 10000  # 有10000個標籤(資料夾)
EPOCHS = 50
LEARNING_RATE = 0.001
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 123456
DATA_DIR = "/content/drive/MyDrive/大專生計畫/data_set/data"  # 資料集路徑

In [61]:
# 確保可重現性
torch.manual_seed(RANDOM_SEED)
# random.seed(RANDOM_SEED)
# np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
  torch.cuda.manual_seed(RANDOM_SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  print('done')
else:
  print('not done')

done


In [62]:
# 定義模型
class UAVLocalizationModel(nn.Module):
  def __init__(self, num_class = NUM_CLASSES):
    super().__init__()
    self.mobilenet = models.mobilenet_v2(pretrained = True)

    for param in self.mobilenet.parameters():
      param.requires_grad = False

    in_feature = self.mobilenet.classifier[1].in_features
    self.mobilenet.classifier = nn.Sequential(
      nn.Dropout(0.2),
      nn.Linear(in_feature, num_class)
    )

  def forward(self, x):
    return self.mobilenet(x)

  def unfreeze_layers(self, num_layers = 5):
    trainable_params = []
    for name, child in self.named_children():
      if name == 'features':
        for i, layer in enumerate(child):
          if i >= len(child) - num_layers:
            for param in layer.parameters():
              param.requires_grad = True
              trainable_params.append(param)
      elif name == 'classifier':
        for param in child.parameters():
          param.requires_grad = True
          trainable_params.append(param)
    return trainable_params

In [63]:
def is_valid_file_func(path):
  return path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))
# 加載數據
def load_data():
  train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  train_dataset = datasets.ImageFolder(
    root = DATA_DIR,
    is_valid_file = is_valid_file_func,
    allow_empty = False,
    transform=train_transforms
  )

  train_loader = DataLoader(
    train_dataset,
    batch_size = BATCH_SIZE,
    num_workers=4
  )

  val_loader = DataLoader(
    train_dataset,
    batch_size = BATCH_SIZE,
    num_workers=4
  )

  return train_loader, val_loader, train_dataset.class_to_idx

In [71]:
# 訓練函數
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=EPOCHS):
  history = {
    'train_loss': [],
    'val_loss': [],
    'train_acc': [],
    'val_acc': []
  }

  best_val_acc = 0.0

  for epoch in range(num_epochs):
    model.train()

    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    running_loss = 0.0
    running_corrects = 0

    train_pbar = tqdm(train_loader, desc=f'Training')

    for inputs, labels in train_pbar:
      optimizer.zero_grad()

      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    train_pbar.set_postfix({'loss': loss.item()})

    history['train_loss'].append(epoch_loss)
    history['train_acc'].append(epoch_acc.cpu().numpy())

    print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    # # 驗證階段
    # model.eval()
    # running_loss = 0.0
    # running_corrects = 0
    # val_pbar = tqdm(val_loader, desc=f'Validation')

    # for inputs, labels in val_pbar:
    #   with torch.set_grad_enabled(False):
    #     outputs = model(inputs)
    #     _, preds = torch.max(outputs, 1)
    #     loss = criterion(outputs, labels)

    #   running_loss += loss.item() * inputs.size(0)
    #   running_corrects += torch.sum(preds == labels.data)

    # val_pbar.set_postfix({'loss': loss.item()})
    # epoch_loss = running_loss / len(val_loader.dataset)
    # epoch_acc = running_corrects.double() / len(val_loader.dataset)

    # print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    # # 記錄歷史
    # history['val_loss'].append(epoch_loss)
    # history['val_acc'].append(epoch_acc.cpu().numpy())

    # 保存最佳模型
    if epoch_acc > best_val_acc:
      best_val_acc = epoch_acc
      torch.save(model.state_dict(), '/content/drive/MyDrive/大專生計畫/AI model/uav_localization_best_model.pth')
      print(f'保存最佳模型，準確率: {epoch_acc:.4f}')

  # 加載最佳模型權重
  model.load_state_dict(torch.load('/content/drive/MyDrive/大專生計畫/AI model/uav_localization_best_model.pth'))

  return model, history

In [65]:
# 評估模型
def evaluate_model(model, test_loader):
  return None

In [66]:
# 繪製訓練曲線
def plot_training_history(history):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

  ax1.plot(history['train_loss'], label='Train Loss')
  ax1.plot(history['val_loss'], label='Val Loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Loss')
  ax1.legend()
  ax1.set_title('Training and Validation Loss')

  ax2.plot(history['train_acc'], label='Train Accuracy')
  ax2.plot(history['val_acc'], label='Val Accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.legend()
  ax2.set_title('Training and Validation Accuracy')

  plt.tight_layout()
  plt.savefig('training_history.png')
  plt.show()

In [67]:
print("載入數據...")
train_loader, val_loader, class_to_idx = load_data()

載入數據...


In [68]:
print("創建模型...")
model = UAVLocalizationModel(NUM_CLASSES)

創建模型...


In [69]:
!nvidia-smi
if torch.cuda.is_available():
  print("GPU is available")
else:
  print("GPU is not available")


Thu Apr  3 10:32:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 第一階段：只訓練分類器層
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mobilenet.classifier.parameters(), lr=LEARNING_RATE)
print("開始第一階段訓練...")
model, first_history = train_model(
  model,
  train_loader,
  val_loader,
  criterion,
  optimizer,
  num_epochs=EPOCHS
)

開始第一階段訓練...
Epoch 1/50
----------


Training: 100%|██████████| 313/313 [14:28<00:00,  2.77s/it]


Training Loss: 7.9487 Acc: 0.1160
保存最佳模型，準確率: 0.1160
Epoch 2/50
----------


Training:  96%|█████████▌| 299/313 [13:43<00:37,  2.71s/it]